In [ ]:
!unzip '/content/emotions.zip'

Archive:  /content/emotions.zip
replace test.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd

In [ ]:
df_train = pd.read_csv("train.txt",delimiter=';',names=['text','label'])
df_val = pd.read_csv("val.txt",delimiter=';',names=['text','label'])

In [ ]:
df = pd.concat([df_train,df_val])

In [ ]:
pd.set_option('display.max_colwidth', None)
df.head()

,text,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplace i will know that it is still on the property,love
4,i am feeling grouchy,anger


In [ ]:
df[15998:16005]

,text,label
15998,i feel like this was such a rude comment and im glad that t,anger
15999,i know a lot but i feel so stupid because i can not portray it,sadness
0,im feeling quite sad and sorry for myself but ill snap out of it soon,sadness
1,i feel like i am still looking at a blank canvas blank pieces of paper,sadness
2,i feel like a faithful servant,love
3,i am just feeling cranky and blue,anger
4,i can have for a treat or if i am feeling festive,joy


In [ ]:
df.reset_index(inplace=True,drop=True)

In [ ]:
df[15998:16005]

,text,label
15998,i feel like this was such a rude comment and im glad that t,anger
15999,i know a lot but i feel so stupid because i can not portray it,sadness
16000,im feeling quite sad and sorry for myself but ill snap out of it soon,sadness
16001,i feel like i am still looking at a blank canvas blank pieces of paper,sadness
16002,i feel like a faithful servant,love
16003,i am just feeling cranky and blue,anger
16004,i can have for a treat or if i am feeling festive,joy


In [ ]:
df.shape

(18000, 2)

In [ ]:
df.label.value_counts()

joy         6066
sadness     5216
anger       2434
fear        2149
love        1482
surprise     653
Name: label, dtype: int64

In [ ]:
def custom_encoder(df):
  df.replace(to_replace=["joy", "love", "surprise"], value=1, inplace=True)
  df.replace(to_replace=["sadness", "anger", "fear"], value=0, inplace=True)
  return df

In [ ]:
df.label = custom_encoder(df.label)

In [ ]:
df.head(10)

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake,0
2,im grabbing a minute to post i feel greedy wrong,0
3,i am ever feeling nostalgic about the fireplace i will know that it is still on the property,1
4,i am feeling grouchy,0
5,ive been feeling a little burdened lately wasnt sure why that was,0
6,ive been taking or milligrams or times recommended amount and ive fallen asleep a lot faster but i also feel like so funny,1
7,i feel as confused about life as a teenager or as jaded as a year old man,0
8,i have been with petronas for years i feel that petronas has performed well and made a huge profit,1
9,i feel romantic too,1


In [ ]:
df.label.value_counts()

0    9799
1    8201
Name: label, dtype: int64

# **Data Pre-processing**

In [ ]:
import string
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

In [ ]:
import nltk
nltk.download('punkt')
def tokenization(text):
    words = nltk.word_tokenize(text)
    return words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatizer(text):
  lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
  return lemm_text

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
def preprocess(df_col):
  corpus = []
  for item in df_col:
    new_item = remove_punctuation(item)
    new_item = new_item.lower()
    new_item = tokenization(new_item)
    new_item = remove_stopwords(new_item)
    new_item = lemmatizer(new_item)
    corpus.append(' '.join(str(x) for x in new_item))
  return corpus

In [ ]:
corpus = preprocess(df.text)

In [ ]:
corpus[0:10]

['didnt feel humiliated',
 'go feeling hopeless damned hopeful around someone care awake',
 'im grabbing minute post feel greedy wrong',
 'ever feeling nostalgic fireplace know still property',
 'feeling grouchy',
 'ive feeling little burdened lately wasnt sure',
 'ive taking milligram time recommended amount ive fallen asleep lot faster also feel like funny',
 'feel confused life teenager jaded year old man',
 'petronas year feel petronas performed well made huge profit',
 'feel romantic']

In [ ]:
df.label.head(10)

0    0
1    0
2    0
3    1
4    0
5    0
6    1
7    0
8    1
9    1
Name: label, dtype: int64

# **Bag-of-words**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1,2))
traindata = cv.fit_transform(corpus)
X = traindata
y = df.label

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 100)
clf.fit(X, y)

RandomForestClassifier()

In [ ]:
from sklearn import metrics
y_pred = clf.predict(X) 
metrics.accuracy_score(y_pred, y)

0.9996111111111111

# **Testing**

In [ ]:
test_df = pd.read_csv('test.txt',delimiter=';',names=['text','label'])

In [ ]:
test_df.head()

,text,label
0,im feeling rather rotten so im not very ambitious right now,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i don t ever want her to feel like i m ashamed with her,sadness
3,i left with my bouquet of red and yellow tulips under my arm feeling slightly more optimistic than when i arrived,joy
4,i was feeling a little vain when i did this one,sadness


In [ ]:
X_test,y_test = test_df.text,test_df.label

In [ ]:
test_label = custom_encoder(y_test)

In [ ]:
test_label.head()

0    0
1    0
2    0
3    1
4    0
Name: label, dtype: int64

In [ ]:
test_corpus = preprocess(X_test)

In [ ]:
test_corpus[0:5]

['im feeling rather rotten im ambitious right',
 'im updating blog feel shitty',
 'never make separate ever want feel like ashamed',
 'left bouquet red yellow tulip arm feeling slightly optimistic arrived',
 'feeling little vain one']

In [ ]:
testdata = cv.transform(test_corpus)

In [ ]:
predictions = clf.predict(testdata)

In [ ]:
metrics.accuracy_score(y_test,predictions)

0.962

In [ ]:
def find_sentiment(input):
  input = cv.transform(preprocess(input))
  prediction = clf.predict(input)
  if prediction == 0: 
    print('Input statement has Negative sentiment')
  if prediction == 1:
    print('Input statement has Positive sentiment')

In [ ]:
input = ["Sometimes I just want to punch someone in the face."]
find_sentiment(input)

Input statement has Negative sentiment


In [ ]:
input = ["I bought a new phone and it's so good."]
find_sentiment(input)

Input statement has Positive sentiment


In [ ]:
input = ["i am feeling extremely sad"]
find_sentiment(input)

Input statement has Negative sentiment


In [ ]:
input = ["Game of Thrones is an amazing tv series!"]
find_sentiment(input)

Input statement has Positive sentiment
